# Домашнее задание №7. Сверточные нейронные сети для анализа текста

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

from string import punctuation
from stop_words import get_stop_words
from pymorphy2 import MorphAnalyzer
import re

import nltk
from nltk.tokenize import word_tokenize
nltk.download("punkt")
from nltk.probability import FreqDist

import numpy as np
import tensorflow as tf
import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Input, Embedding, Conv1D, GlobalMaxPool1D
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import TensorBoard 
from keras.losses import SparseCategoricalCrossentropy
from keras.callbacks import EarlyStopping  

from sklearn.preprocessing import LabelEncoder
import gensim

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\BazhanovaEN\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
#функция подсчета f1_score
import keras.backend as K
def get_f1(y_true, y_pred): #taken from old keras source code
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val

## Заданиe
Берем отызывы за лето (из архива с материалами или предыдущего занятия)
1. Учим conv сеть для классификации
2. Рассмотреть 2-а варианта сеточек
2.1 Инициализировать tf.keras.layers.Embedding предобученными векторами взять к примеру с https://rusvectores.org/ru/
2.2 Инициализировать слой tf.keras.layers.Embedding по умолчанию (ну то есть вам ничего не делать с весами)

Сравнить две архитектуры с предобученными весами и когда tf.keras.layers.Embedding обучается сразу со всей сеточкой, что получилось лучше

## План решения

[0. Загрузка и просмотр данных](#section_0)

[1. Предобработка данных](#section_1)

[2. Keras CONV модель с Embedding слоем](#section_2)

[2.1. Keras CONV модель с Embedding слоем по умолчанию](#section_2.1)

[2.2. Keras CONV модель с предобученным Embedding слоем](#section_2.2)

[3. Выводы](#section_3)

## Загрузка и просмотр данных  <a id='section_0'></a>

In [3]:
df = pd.read_excel('отзывы за лето.xls')
df.head(10)

,Rating,Content,Date
0,5,It just works!,2017-08-14
1,4,В целом удобноное приложение...из минусов хотя...,2017-08-14
2,5,Отлично все,2017-08-14
3,5,Стал зависать на 1% работы антивируса. Дальше ...,2017-08-14
4,5,"Очень удобно, работает быстро.",2017-08-14
5,5,Всё удобно норм 👍👍👍,2017-08-14
6,5,Очень удобное приложение.,2017-08-14
7,5,Все устраивает,2017-08-14
8,5,У меня работает все четко. В отличии от банком...,2017-08-14
9,5,Очень все хорошо👍,2017-08-14


In [4]:
df['Rating'].value_counts()

5    14586
1     2276
4     2138
3      911
2      748
Name: Rating, dtype: int64

In [5]:
df_train, df_test = train_test_split(df, test_size=0.33, random_state=42)
df_train = df_train.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)
df_train.shape, df_test.shape

((13841, 3), (6818, 3))

In [6]:
df_train

,Rating,Content,Date
0,5,Наконец-то исправили эту чушь с неоргинальной ...,2017-08-09
1,5,Удобно в использовании,2017-07-27
2,5,Отлично,2017-08-08
3,5,Класс,2017-07-25
4,5,Удобно,2017-07-08
...,...,...,...
13836,4,Все нравится,2017-07-29
13837,5,Очень смешно программа пугается рута :),2017-07-28
13838,1,Не могу скачать ошибка номер 24,2017-08-06
13839,5,Сбербанк всегда рядом,2017-08-12


## 1. Предобработка данных <a id='section_1'></a>

In [7]:
sw = set(get_stop_words("ru"))
exclude = set(punctuation)
morpher = MorphAnalyzer()

def preprocess_text(txt):
    txt = str(txt)
    txt = "".join(c for c in txt if c not in exclude)
    txt = txt.lower()
    txt = re.sub("\sне", "не", txt)
    txt = [morpher.parse(word)[0].normal_form for word in txt.split() if word not in sw]
    return " ".join(txt)

df_train['Content'] = df_train['Content'].apply(preprocess_text)
df_test['Content'] = df_test['Content'].apply(preprocess_text)

In [8]:
df_train.head()

,Rating,Content,Date
0,5,наконецтый исправить чушь снеоргинальный проши...,2017-08-09
1,5,удобно использование,2017-07-27
2,5,отлично,2017-08-08
3,5,класс,2017-07-25
4,5,удобно,2017-07-08


**Готовим словарь**

In [9]:
train_corpus = " ".join(df_train["Content"])
train_corpus = train_corpus.lower()
train_corpus

'наконецтый исправить чушь снеоргинальный прошивка приложение удобно пользоваться удобно использование отлично класс удобно замечательный приложение довольный влиять работа устройство приходиться периодически отключать оплатить мобильный приложение коммунальный услуга гораздо простой ноут шаблон проводиться клик удобно сказать пропасть переводить другой клиент сбербанк оплачивать чужой мобильный телефон программа серьёзно поработать отлично смело качать устраивать принцип хороший программа удобно быстро обновление сталон удобно делать выписка счёт тк увидеть поступление сегодняшний число происходить день просить разработчик обратить особый внимание удобно норма отлично антивирус клавиатура пятизначный пин отлично пойти класс asus ze551ml клёво работать антивирус справляться 10 сечь довольный пятёрочка  быстро удобно сбербанк онлайн удобство комфорт хороший приложение правда многий функциямин пользоваться нравиться разработчик удобно нравиться норма нравиться заходить какуюнеделюисправи

In [10]:
tokens = word_tokenize(train_corpus)
tokens

['наконецтый',
 'исправить',
 'чушь',
 'снеоргинальный',
 'прошивка',
 'приложение',
 'удобно',
 'пользоваться',
 'удобно',
 'использование',
 'отлично',
 'класс',
 'удобно',
 'замечательный',
 'приложение',
 'довольный',
 'влиять',
 'работа',
 'устройство',
 'приходиться',
 'периодически',
 'отключать',
 'оплатить',
 'мобильный',
 'приложение',
 'коммунальный',
 'услуга',
 'гораздо',
 'простой',
 'ноут',
 'шаблон',
 'проводиться',
 'клик',
 'удобно',
 'сказать',
 'пропасть',
 'переводить',
 'другой',
 'клиент',
 'сбербанк',
 'оплачивать',
 'чужой',
 'мобильный',
 'телефон',
 'программа',
 'серьёзно',
 'поработать',
 'отлично',
 'смело',
 'качать',
 'устраивать',
 'принцип',
 'хороший',
 'программа',
 'удобно',
 'быстро',
 'обновление',
 'сталон',
 'удобно',
 'делать',
 'выписка',
 'счёт',
 'тк',
 'увидеть',
 'поступление',
 'сегодняшний',
 'число',
 'происходить',
 'день',
 'просить',
 'разработчик',
 'обратить',
 'особый',
 'внимание',
 'удобно',
 'норма',
 'отлично',
 'антивирус',
 

Отфильтруем и возьмем только топ N токенов

In [11]:
tokens_filtered = [word for word in tokens if word.isalnum()] #фильтруем, чтобы остались только буквы и цифры

#создаем словарь токен:количество в корпусе
dist = FreqDist(tokens_filtered)
dist

FreqDist({'приложение': 4123, 'удобно': 2201, 'работать': 1288, 'удобный': 1182, 'отлично': 860, 'нравиться': 763, 'хороший': 681, 'отличный': 677, 'телефон': 627, 'супер': 540, ...})

In [12]:
max_words = 200

tokens_filtered_top = [pair[0] for pair in dist.most_common(max_words-1)]
tokens_filtered_top[:10]

['приложение',
 'удобно',
 'работать',
 'удобный',
 'отлично',
 'нравиться',
 'хороший',
 'отличный',
 'телефон',
 'супер']

In [13]:
#создаем словарь из топ-200 токенов: токен:его рейтинг(место в топ-200)
vocabulary = {v: k for k, v in dict(enumerate(tokens_filtered_top, 1)).items()}
vocabulary

{'приложение': 1,
 'удобно': 2,
 'работать': 3,
 'удобный': 4,
 'отлично': 5,
 'нравиться': 6,
 'хороший': 7,
 'отличный': 8,
 'телефон': 9,
 'супер': 10,
 'быстро': 11,
 'обновление': 12,
 'пароль': 13,
 'мочь': 14,
 'пользоваться': 15,
 'антивирус': 16,
 'банк': 17,
 'вход': 18,
 'устраивать': 19,
 'сбербанк': 20,
 'раз': 21,
 'прошивка': 22,
 'карта': 23,
 'проблема': 24,
 'рута': 25,
 'программа': 26,
 'ошибка': 27,
 'разработчик': 28,
 'сделать': 29,
 'приходиться': 30,
 'вводить': 31,
 'перевод': 32,
 'счёт': 33,
 'писать': 34,
 'норма': 35,
 'деньга': 36,
 'довольный': 37,
 'около': 38,
 'постоянно': 39,
 'нормально': 40,
 'код': 41,
 'исправить': 42,
 'смс': 43,
 'платёж': 44,
 'понятно': 45,
 'последний': 46,
 'функция': 47,
 'зайти': 48,
 'свой': 49,
 'вылетать': 50,
 'мобильный': 51,
 'стать': 52,
 'шаблон': 53,
 'приходить': 54,
 'возможность': 55,
 'право': 56,
 'делать': 57,
 'иня': 58,
 'проверка': 59,
 'класс': 60,
 'установить': 61,
 'root': 62,
 'заходить': 63,
 '5': 

## 2. Keras CONV модель с Embedding слоем <a id='section_2'></a>

Подготовим Х и у к обучению. На входной Embedding слой модели (x_train и x_test) поступают положительные целые числа.

In [14]:
max_len = 40

#преобразуем трейновый и тестовый датасеты:
#приводим к нижнему регистру, токенизируем, удаляем символы нецифр и небукв
#проверяем: если токен есть в словаре, то в результат, пишет его номер из словаря
           #в противном случае -- пропускаем, до полной длины добиваем нулями
def text_to_sequence(text, maxlen):
    result = []
    tokens = word_tokenize(text.lower())
    tokens_filtered = [word for word in tokens if word.isalnum()]
    for word in tokens_filtered:
        if word in vocabulary:
            result.append(vocabulary[word])
    padding = [0]*(maxlen-len(result))
    return padding + result[-maxlen:]

In [15]:
x_train = np.asarray([text_to_sequence(text, max_len) for text in df_train["Content"]], dtype=np.int32)
x_test = np.asarray([text_to_sequence(text, max_len) for text in df_test["Content"]], dtype=np.int32)

In [16]:
x_train

array([[  0,   0,   0, ...,   1,   2,  15],
       [  0,   0,   0, ...,   0,   2, 181],
       [  0,   0,   0, ...,   0,   0,   5],
       ...,
       [  0,   0,   0, ..., 164,  27,  84],
       [  0,   0,   0, ...,   0,   0,  20],
       [  0,   0,   0, ...,   0, 113,   5]])

Для подсчета функции потерь y_train и y_test должны быть представлены one-hot кодированием.

In [18]:
le = LabelEncoder()
train_enc_labels = le.fit_transform(df_train['Rating']) 
test_enc_labels = le.transform(df_test['Rating'])
le.classes_

array([1, 2, 3, 4, 5], dtype=int64)

In [19]:
train_enc_labels

array([4, 4, 4, ..., 0, 4, 4], dtype=int64)

In [20]:
num_classes = 5
y_train = tf.keras.utils.to_categorical(train_enc_labels, num_classes=num_classes)
y_test = tf.keras.utils.to_categorical(test_enc_labels, num_classes=num_classes)
y_train

array([[0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.],
       ...,
       [1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.]], dtype=float32)

### 2.1. Keras CONV модель с Embedding слоем по умолчанию <a id='section_2.1'></a>

In [21]:
model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=128, input_length=max_len)) 
                    #inputdim -- размер словаря, outputdim -- длина вектора, input_length -- длина входной последовательности
                    #на вход: (батч, inputlen), на выходе: (батч, inputlen, outputdim)
model.add(Conv1D(128, 3))
                   #128 -- длина 1D-фильтра, шаг -- 3
                   #на выходе ([128/3], 128)
model.add(Activation("relu"))
                    #применяем функцию активации к выходу предыдущего слоя
                    #на выходе ([128/3], 128)
model.add(GlobalMaxPool1D())
                    #в каждой свертке оставляет максимальный элемент
                    #на выходе ([128/3], 1)
model.add(Dense(10))
               #10-количество выходов
model.add(Activation("relu"))
model.add(Dense(num_classes))
                #num_classes = 5 -- количество выходов
model.add(Activation('softmax'))
                #преобразуем вектор в рапределение вероятностей

In [22]:
model.compile(loss='categorical_crossentropy', #y_pred должен быть распределением вероятностей, y_true -- one-hot кодированный тензор
              optimizer='adam',
              metrics=[get_f1])

In [23]:
tensorboard=TensorBoard(log_dir='./logs', write_graph=True, write_images=True)
early_stopping=EarlyStopping(monitor='val_loss')  

epochs = 20
batch_size = 512

history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[tensorboard, early_stopping])

Epoch 1/20
25/25 [==============================] - 2s 74ms/step - loss: 1.4763 - get_f1: 0.0058 - val_loss: 1.0751 - val_get_f1: 0.7145
Epoch 2/20
25/25 [==============================] - 2s 61ms/step - loss: 1.0191 - get_f1: 0.7111 - val_loss: 0.8198 - val_get_f1: 0.7435
Epoch 3/20
25/25 [==============================] - 2s 61ms/step - loss: 0.7816 - get_f1: 0.7550 - val_loss: 0.7495 - val_get_f1: 0.7541
Epoch 4/20
25/25 [==============================] - 1s 59ms/step - loss: 0.6759 - get_f1: 0.7803 - val_loss: 0.7164 - val_get_f1: 0.7551
Epoch 5/20
25/25 [==============================] - 1s 59ms/step - loss: 0.6752 - get_f1: 0.7729 - val_loss: 0.7048 - val_get_f1: 0.7545
Epoch 6/20
25/25 [==============================] - 2s 60ms/step - loss: 0.6489 - get_f1: 0.7835 - val_loss: 0.7022 - val_get_f1: 0.7560
Epoch 7/20
25/25 [==============================] - 1s 60ms/step - loss: 0.6589 - get_f1: 0.7742 - val_loss: 0.6999 - val_get_f1: 0.7560
Epoch 8/20
25/25 [=======================

In [24]:
score = model.evaluate(x_test, y_test, batch_size=batch_size, verbose=1)
print('\n')
print('Test loss:', score[0])
print('Test f1_score:', score[1])

14/14 [==============================] - 0s 15ms/step - loss: 0.6655 - get_f1: 0.7755


Test loss: 0.6654693484306335
Test f1_score: 0.7755452990531921


### 2.2. Keras CONV модель с предобученным Embedding слоем  <a id='section_2.2'></a>

In [25]:
#загрузим векторы модели ruwikiruscorpora_upos_cbow_300_10_2021 (https://rusvectores.org/ru/models/)
word_vectors = gensim.models.KeyedVectors.load_word2vec_format('./glove/model.bin', binary=True)  
len(word_vectors), len(word_vectors[1]), word_vectors[1]

(249333,
 300,
 array([ 4.5154796e+00,  3.8297811e+00, -6.9469607e-01, -1.3958697e+00,
        -6.8803182e+00,  2.5444067e+00, -1.3319516e+00, -1.0333906e+00,
        -5.6126447e+00,  7.4659262e+00,  1.4538134e-02, -3.5751505e+00,
         3.5980268e+00, -3.0855367e+00, -4.0427369e-01,  3.5216520e+00,
        -5.8490925e+00, -3.0425388e-01,  1.6812118e+00,  2.9846013e+00,
         1.7240332e+00,  3.9353080e+00,  4.0222373e+00, -1.1100594e+00,
        -1.5550443e+00, -2.1131717e-01, -1.0955086e+00, -5.9671655e+00,
         1.1800621e-01, -6.5576923e-01,  4.9077687e+00,  2.3885634e+00,
         1.3892661e+00,  3.8770003e+00,  1.4573048e+00, -8.8545698e-01,
        -7.6731639e+00, -1.4003915e+00, -9.4185764e-01,  1.2374935e+00,
        -6.8622670e+00, -1.0298090e+00, -3.9164896e+00,  2.3562472e+00,
        -2.6401494e+00, -7.1391735e+00, -5.8870583e+00, -4.6986256e+00,
         2.3890433e+00,  6.7100887e+00, -2.9966933e-01,  4.6237164e+00,
         1.9630557e+00,  1.0540729e+00, -3.987162

In [26]:
#обрежем word_vectors по размеру эмбеддингов предыдущей задачи
word_vectors_matrix = [word_vectors[i][:128] for i in range(200)]
word_vectors_matrix

[array([-5.775997  , -1.4856625 ,  1.4282805 ,  5.107271  ,  0.94274306,
         1.633782  ,  2.03106   , -3.862224  ,  7.6835074 , -2.452829  ,
        -1.1344614 , -2.9715064 , -1.1943372 ,  0.60422194, -3.600998  ,
        -3.7676146 ,  2.9703367 , -1.9335803 ,  0.7025744 , -4.9147015 ,
         6.821634  ,  0.5059877 ,  0.41100878,  3.0133476 , -0.8700064 ,
        -2.0856552 ,  6.383491  , -2.8943033 ,  4.720144  ,  3.638207  ,
        -6.6306615 ,  1.812578  , -1.9579812 , -7.607699  , -2.3855708 ,
         1.3066907 ,  4.9606256 , -3.437113  , -0.3333033 ,  4.795064  ,
        -4.8095465 ,  2.3499146 ,  4.8853273 ,  4.687315  ,  0.8327079 ,
         2.8774421 , -4.291407  , -5.108345  , -2.148041  , -4.7741423 ,
         1.0339713 , -1.7400155 ,  0.31231344, -6.4322696 ,  1.6250391 ,
         0.72703356, -0.49629834, -3.1099138 , -2.9617152 ,  2.7351406 ,
        -0.5895083 ,  0.42098752,  3.5558884 , -1.084687  ,  0.96802926,
        -0.8950529 , -1.6569282 ,  4.6770945 ,  0.8

In [27]:
#инициализируем веса в эмбеддинге
initializer = tf.keras.initializers.Constant(word_vectors_matrix)

model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=128, embeddings_initializer =initializer, input_length=max_len))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(GlobalMaxPool1D())
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

In [28]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=[get_f1])

In [29]:
tensorboard=TensorBoard(log_dir='./logs', write_graph=True, write_images=True)
early_stopping=EarlyStopping(monitor='val_loss')  

epochs = 20
batch_size = 512

history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[tensorboard, early_stopping])

Epoch 1/20
25/25 [==============================] - 2s 74ms/step - loss: 1.4093 - get_f1: 0.4511 - val_loss: 0.9621 - val_get_f1: 0.7115
Epoch 2/20
25/25 [==============================] - 2s 62ms/step - loss: 0.9352 - get_f1: 0.7137 - val_loss: 0.8641 - val_get_f1: 0.7214
Epoch 3/20
25/25 [==============================] - 2s 63ms/step - loss: 0.8232 - get_f1: 0.7354 - val_loss: 0.8090 - val_get_f1: 0.7348
Epoch 4/20
25/25 [==============================] - 2s 62ms/step - loss: 0.7803 - get_f1: 0.7414 - val_loss: 0.7875 - val_get_f1: 0.7406
Epoch 5/20
25/25 [==============================] - 2s 61ms/step - loss: 0.7394 - get_f1: 0.7560 - val_loss: 0.7898 - val_get_f1: 0.7425


In [30]:
score = model.evaluate(x_test, y_test, batch_size=batch_size, verbose=1)
print('\n')
print('Test loss:', score[0])
print('Test f1_score:', score[1])

14/14 [==============================] - 0s 15ms/step - loss: 0.7569 - get_f1: 0.7574


Test loss: 0.7569317817687988
Test f1_score: 0.7574301958084106


## 3. Выводы  <a id='section_3'></a>

Нейронная сеть с ембеддингом без инициализации (f1_score = 0,776) дает лучшие показатели метрики, чем сеть с инициализированным эмбеддингом (f1_score = 0,757).